In [1]:

import os
import numpy as np
import pandas as pd
os.chdir('/home/exlysha/MMRec/mydata/Baby')
os.getcwd()
i_id, desc_str = 'itemID', 'description'

file_path = './'
file_name = 'meta-sports14.csv'

meta_file = os.path.join(file_path, file_name)

df = pd.read_csv(meta_file)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (7050, 10)


,itemID,asin,categories,description,title,price,imUrl,brand,related,salesRank
0,0,097293751X,[['Baby']],Easily keep track of your baby's or child's da...,"Baby Tracker&reg; - Daily Childcare Journal, S...",17.00,http://ecx.images-amazon.com/images/I/41Bb6wf%...,Time Too,"{'also_bought': ['9729375011', 'B004FN1AE8', '...",NaN
1,1,9729375011,[['Baby']],This is version of the award-winningBaby Track...,Newborn Baby Tracker&reg; - Round the Clock Ch...,15.95,http://ecx.images-amazon.com/images/I/51r3BLpL...,NaN,"{'also_bought': ['B000V5KPZ4', 'B001F8TLLU', '...",NaN
2,2,B00000IZQI,[['Baby']],This colorful car collection develops motor sk...,Fisher Price Nesting Action Vehicles,8.37,http://ecx.images-amazon.com/images/I/51E83QCC...,NaN,"{'also_bought': ['B0042D69W4', 'B00428LIZM', '...",NaN


In [2]:
# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['description'].isnull() & df['title'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

df[desc_str] = df[desc_str].fillna(" ")
df['title'] = df['title'].fillna(" ")
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")

sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

sentences[:10]

(13, 10)
(664, 10)
(13, 10)
(13, 10)
(0, 10)


["Baby Tracker&reg; - Daily Childcare Journal, Schedule Log Time Too Baby Easily keep track of your baby's or child's daily schedules, activities and needs in one handy spot.  And make the drop-off and pick-up process for childcare a snap.  The Baby Tracker Daily Childcare Journal is designed to make it fast and easy to record and review meals, naps, activities, playtime, daily news, milestones and to-dos ---all in a single page view.  Get ready for doctor visits, give to childcare helpers and use this easy at-a-glance daily record to monitor schedules and prep for your baby's daily needs.  This version records daily activities from 6 a.m. to 7 p.m. and creates a visual snapshot of activities by time each day.   The Baby Tracker Daily Childcare Journal is ready to keep track of up to 6 months of daily schedule tracking (180 days) and is spiral bound to make it portable and lay flat for easy recording on a countertop or nightstand.Each journal includes:*180 daily at-a-glance tracking re

In [3]:
len(sentences)

7050

In [4]:
basic_prompt = "Please perform the following transformation on the product descrn below: 1. If the produciptiot description exceeds 70 tokens, output a shortened version with fewer than 70 tokens.2. If the product description contains 70 tokens or fewer, output the natural language version as-is.3. If the product description is empty, output 'none'.Only output the transformed description, do not include any additional text. Here is the product description:"
results = []
from openai import OpenAI

client = OpenAI(api_key="sk-xqulaqciaqusjymbzvnogkazghejqwicejpnbqxclwliulsz", 
                base_url="https://api.siliconflow.cn/v1")
from tqdm import tqdm
for sentence in tqdm(sentences, desc="Processing"):
    prompt = basic_prompt+sentence
    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3",
        messages=[
            {'role': 'user', 
            'content': prompt}
        ],
        stream=True
    )
    output = ""
    for chunk in response:
        if chunk.choices[0].delta.content:
            output += chunk.choices[0].delta.content
    results.append(output)

Processing: 100%|██████████| 7050/7050 [5:06:54<00:00,  2.61s/it]  


In [5]:
results

["Baby Tracker&reg; helps track your baby's daily schedules, activities, and needs in one journal. Record meals, naps, playtime, and milestones easily. Includes 180 days of tracking, emergency forms, and immunization records. Ideal for caregivers and parents. Award-winning design.",
 'Newborn Baby Tracker&reg; - A round-the-clock journal to monitor newborns and nursing moms with easy tracking of feedings, diaper changes, and sleep. Spiral-bound with daily notes and parent pages. Fits 3 months of tracking. Award-winning design.',
 'Fisher Price Nesting Action Vehicles help develop motor skills with four colorful, stackable cars that nest in order. Includes sports car, police car, school bus, and fire truck.',
 'My Quiet Book, Fabric Activity Book for Children. This cloth book helps kids recognize shapes, tell time, and practice buttoning, buckling, snapping, lacing, and zipping. It features bright activities, movable clock hands, and textured fabrics. Great for learning and travel.',
 '

In [6]:
assert len(results) == len(sentences)

In [7]:
results[:10]

["Baby Tracker&reg; helps track your baby's daily schedules, activities, and needs in one journal. Record meals, naps, playtime, and milestones easily. Includes 180 days of tracking, emergency forms, and immunization records. Ideal for caregivers and parents. Award-winning design.",
 'Newborn Baby Tracker&reg; - A round-the-clock journal to monitor newborns and nursing moms with easy tracking of feedings, diaper changes, and sleep. Spiral-bound with daily notes and parent pages. Fits 3 months of tracking. Award-winning design.',
 'Fisher Price Nesting Action Vehicles help develop motor skills with four colorful, stackable cars that nest in order. Includes sports car, police car, school bus, and fire truck.',
 'My Quiet Book, Fabric Activity Book for Children. This cloth book helps kids recognize shapes, tell time, and practice buttoning, buckling, snapping, lacing, and zipping. It features bright activities, movable clock hands, and textured fabrics. Great for learning and travel.',
 '

In [8]:
df['results'] = results
df.to_csv(meta_file, index=False)